In [53]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn import linear_model
from sklearn.cross_validation import KFold

def importAndClean():
    df_train = pd.read_csv("HCMST_train_clean.csv", header=0, low_memory=False)
    df_test = pd.read_csv("HCMST_test_clean.csv", header=0, low_memory=False)
        
    x_train = df_train
    x_train = x_train.drop(x_train.columns[[0,1,2,118,119]], axis = 1).values
    
    y_train = df_train['Breakup binary'].values

    x_test = df_test
    x_test = x_test.drop(x_test.columns[[0,1,2,118,119]], axis=1).values
    y_test = df_test['Breakup binary'].values

    return x_train, y_train, x_test, y_test

def applyClassifier(type,x,y,x_test):
    # Fit and Predict on New Data
    fold = KFold(len(y), n_folds=5, shuffle=True, random_state=777)
    if type == 'logistic':
        regr = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10)))
        ,penalty='l2'
        ,scoring='roc_auc'
        ,cv=fold
        ,random_state=777
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='newton-cg'
        ,tol=10)
        pred = regr.fit(x,y).predict(x_test)    
        return pred
    elif type == 'lasso':
        clf = linear_model.Lasso(alpha=0.01)
        pred = clf.fit(x,y).predict(x_test)
#         print (clf.coef_)
#         print (clf.intercept_)
        return pred

def main():
    x_train, y_train, x_test, y_test = importAndClean()

#     pred_rf = applyClassifier('forest', x_train, y_train, x_test)
    pred_log = applyClassifier('logistic', x_train, y_train, x_test)
    pred_lasso = applyClassifier('lasso', x_train, y_train, x_test)
    pred_lasso[pred_lasso >= 0.5] = 1
    pred_lasso[pred_lasso < 0.5] = 0
    
#     print("Classifier Accuracy: %.2f" % np.mean(y_test == pred_mc))
#     print("Classifier Accuracy: %.2f" % np.mean(y_test == pred_rf))
    print("Logistic Regression Classifier Error Rate: %.5f" % np.mean(y_test != pred_log))
    print("Lasso Regression Error Rate: %.5f" % np.mean(y_test != pred_lasso))
    

if __name__ == "__main__":
    main()



Logistic Regression Classifier Error Rate: 0.15591
Lasso Regression Error Rate: 0.17025
